# Basic Samples : Agtype mapper for psycopg driver

You can make transactions and queries for PostgreSQL with Psycopg.

This module enable to mapping agtype to python class(Path, Vertex, Edge)

## Connect to PostgreSQL and agType setting

In [21]:
import psycopg 
import age

GRAPH_NAME = "test_graph"
ag = age.connect(host="172.17.0.2", port="5432", dbname="postgre", user="postgres", password="agens", graph=GRAPH_NAME)
conn = ag.connection


In [22]:
with conn.cursor() as cursor:
    try :
        cursor.execute("""SELECT * from cypher(%s, $$ CREATE (n:Person {name: 'Joe', title: 'Developer'}) $$) as (v agtype); """, (GRAPH_NAME,) )
        cursor.execute("""SELECT * from cypher(%s, $$ CREATE (n:Person {name: 'Smith', title: 'Developer'}) $$) as (v agtype); """, (GRAPH_NAME,))
        cursor.execute("""SELECT * from cypher(%s, $$ 
            CREATE (n:Person {name: 'Tom', title: 'Manager'}) 
            RETURN n
            $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            print("CREATED::", row[0])
        
        
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Smith'}) CREATE (a)-[r:workWith {weight: 5}]->(b)
            $$) as (v agtype); """, (GRAPH_NAME,))
        
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH (a:Person {name: 'Smith'}), (b:Person {name: 'Tom'}) CREATE (a)-[r:workWith {weight: 3}]->(b)
            $$) as (v agtype); """, (GRAPH_NAME,))
        
        # When data inserted or updated, You must commit.
        conn.commit()
    except Exception as ex:
        print(type(ex), ex)
        # if exception occurs, you must rollback all transaction. 
        conn.rollback()

with conn.cursor() as cursor:
    try:
        print("------- [Select Vertices] --------")
        cursor.execute("""SELECT * from cypher(%s, $$ MATCH (n) RETURN n $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            vertex = row[0]
            print(vertex.id, vertex.label, vertex["name"], vertex["title"])
            print("-->", vertex)
            
        print(type(cursor))
        print("------- [Select Paths] --------")
        cursor.execute("""SELECT * from cypher(%s, $$ MATCH p=()-[]->() RETURN p LIMIT 10 $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            path = row[0]
            v1 = path[0]
            e1 = path[1]
            v2 = path[2]
            print(v1.gtype , v1["name"], e1.gtype , e1.label, e1["weight"], v2.gtype , v2["name"])
            print("-->", path)
    except Exception as ex:
        print(type(ex), ex)
        # if exception occurs, you must rollback even though just retrieving.
        conn.rollback()

CREATED:: {label:Person, id:844424930131971, properties:{name: Tom, title: Manager}}::VERTEX
------- [Select Vertices] --------
844424930131969 Person Joe Developer
--> {label:Person, id:844424930131969, properties:{name: Joe, title: Developer}}::VERTEX
844424930131970 Person Smith Developer
--> {label:Person, id:844424930131970, properties:{name: Smith, title: Developer}}::VERTEX
844424930131971 Person Tom Manager
--> {label:Person, id:844424930131971, properties:{name: Tom, title: Manager}}::VERTEX
<class 'psycopg.ClientCursor'>
------- [Select Paths] --------
1 Joe 2 workWith 5 1 Smith
--> [{label:Person, id:844424930131969, properties:{name: Joe, title: Developer}}::VERTEX,{label:workWith, id:1125899906842625, properties:{weight: 5}, start_id:844424930131969, end_id:844424930131970}::EDGE,{label:Person, id:844424930131970, properties:{name: Smith, title: Developer}}::VERTEX]::PATH
1 Smith 2 workWith 3 1 Tom
--> [{label:Person, id:844424930131970, properties:{name: Smith, title: Dev

In [23]:
with conn.cursor() as cursor:
    try:
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH p=(a)-[b]->(c) RETURN a.name, label(b), c.name 
            $$) as (a agtype, b agtype, c agtype); """, (GRAPH_NAME,))
        for row in cursor:
            print(row[0], row[1], row[2])
            print("-->", row)
    except Exception as ex:
        print(ex)
        conn.rollback()

Joe workWith Smith
--> ('Joe', 'workWith', 'Smith')
Smith workWith Tom
--> ('Smith', 'workWith', 'Tom')


In [24]:
with conn.cursor() as cursor:
    try :
        cursor.execute("""SELECT * from cypher(%s, $$ 
            CREATE (n:Person {name: 'Jack', title: 'Developer', score:-6.45161290322581e+46}) 
            $$) as (v agtype); """, (GRAPH_NAME,) )
        cursor.execute("""SELECT * from cypher(%s, $$ 
            CREATE (n:Person {name: 'John', title: 'Developer'}) 
            $$) as (v agtype); """, (GRAPH_NAME,))

        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'John'}) 
            CREATE (a)-[r:workWith {weight: 2}]->(b)
            $$) as (v agtype); """, (GRAPH_NAME,))
        
        # When data inserted or updated, You must commit 
        conn.commit()
    except Exception as ex:
        print(ex)
        conn.rollback()

with conn.cursor() as cursor:
    try :
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH p=(a )-[b]->(c) RETURN a , b, c 
            $$) as (ta agtype, tb agtype, tc agtype); """, (GRAPH_NAME,))
        
        for row in cursor:
            print(row[0]["name"], row[1].properties, row[2]["name"])
            
    except Exception as ex:
        print(ex)
        conn.rollback()


Joe {'weight': 5} Smith
Smith {'weight': 3} Tom
Jack {'weight': 2} John


In [25]:
with conn.cursor() as cursor:
    try:
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH p=(a)-[b]->(c) RETURN p  
            $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            path = row[0]
            print(path[0]["name"], path[1].id, path[1].properties, path[2]["name"])
    except Exception as ex:
        print(ex)
        conn.rollback()

Joe 1125899906842625 {'weight': 5} Smith
Smith 1125899906842626 {'weight': 3} Tom
Jack 1125899906842627 {'weight': 2} John


In [26]:
age.deleteGraph(conn, GRAPH_NAME)
conn.close()